In [2]:
%%capture
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
#thanks @keskarnitish

In [3]:

#тут будет текст
corpora = ""

#with open("nietzsche.txt") as fin:
with open("Call_Him_Lord_by_R_Gordon_Dickson.txt") as fin:
    text = fin.read()#.decode('cp1251')
    corpora += text
        



In [4]:
#тут будут все уникальные токены (буквы, цифры)
tokens = set(corpora)

tokens = list(tokens)

In [5]:
len(tokens)

66

In [7]:
token_to_id = {token:id for id, token in enumerate(tokens)}
id_to_token = {id:token for token, id in token_to_id.items()}

#Преобразуем всё в токены
corpora_ids = map(lambda x: token_to_id[x], corpora)

In [8]:
def sample_random_batches(source,n_batches=10, seq_len=20):
    """Функция, которая выбирает случайные тренировочные примеры из корпуса текста в токенизированном формате.
    
    source - массив целых чисел - номеров токенов в корпусе (пример - corpora_ids)
    n_batches - количество случайных подстрок, которые нужно выбрать
    
    seq_len - длина одной подстроки без учёта ответа
    
    
    Вернуть нужно кортеж (X,y), где
    
    X - матрица, в которой каждая строка - подстрока длины [seq_len].
    
    y - вектор, в котором i-тое число - символ следующий в тексте сразу после i-той строки матрицы X
    
    Проще всего для этого сначала создать матрицу из строк длины seq_len+1,
    а потом отпилить от неё последний столбец в y, а все остальные - в X
    
    Если делаете иначе - пожалуйста, убедитесь, что в у попадает правильный символ, ибо позже эту ошибку 
    будет очень тяжело заметить.
    
    Также убедитесь, что ваша функция не вылезает за край текста (самое начало или конец текста).
    
    Следующая клетка проверяет часть этих ошибок, но не все.
    """
    X_batch = []
    y_batch = []
    for i in range(n_batches):
        start = np.random.randint(0,len(source) - (seq_len + 1))
        stop = start + seq_len
        X_batch.append(source[start:stop])
        y_batch.append(source[stop:stop + 1])
    X_batch = np.array(X_batch)
    y_batch = np.array(y_batch)
    return X_batch, y_batch
    
    

# Константы

In [9]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 5

# Максимальный модуль градиента
grad_clip = 100




# Входные переменные

In [10]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')


# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.

In [11]:
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)

nn = lasagne.layers.EmbeddingLayer(l_in, input_size=len(tokens), output_size=128)

nn = lasagne.layers.LSTMLayer(nn, num_units=512, grad_clipping=grad_clip)

nn = lasagne.layers.SliceLayer(nn, -1, 1)

l_out = lasagne.layers.DenseLayer(nn, num_units=len(tokens), nonlinearity=lasagne.nonlinearities.softmax)




In [13]:
# Веса модели
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print(weights)

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b]


In [14]:
network_output = lasagne.layers.get_output(l_out)
#если вы используете дропаут - не забудьте продублировать всё в режиме deterministic=True

In [15]:
loss = T.nnet.categorical_crossentropy(network_output,target_values).mean()

updates = lasagne.updates.rmsprop(loss, weights, learning_rate=0.01)


# Компилируем всякое-разное

In [16]:

#обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)


# Генерируем своего Ницше (или Гордона Диксона)

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

In [17]:
def max_sample_fun(probs):
#     print 'probs'
#     print probs
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    
    
    return np.random.choice(len(probs), 1, p=probs)[0]






In [20]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 




def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print("Using random seed:",seed_phrase)
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    #assert type(seed_phrase) is unicode
        
        
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])
    
    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)


In [21]:
generate_sample(max_sample_fun,None)

Using random seed: and E


TypeError: ('Bad input argument to theano function with name "<ipython-input-16-f8a13d93291b>:9"  at index 0(0-based)', "int() argument must be a string, a bytes-like object or a number, not 'map'")

# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.
Обучение идёт долго, для демонстрации можно прервать раньше, эту модель всё равно можно будет использовать. 


In [24]:

print("Training ...")


#сколько всего эпох
n_epochs=500

# раз в сколько эпох печатать примеры 
batches_per_epoch = 100

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size=10


for epoch in range(n_epochs):

    print("Генерируем текст в пропорциональном режиме")
    generate_sample(proportional_sample_fun,None)
    
    print("Генерируем текст в жадном режиме (наиболее вероятные буквы)")
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y[:,0])
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))



Training ...
Генерируем текст в пропорциональном режиме
Using random seed: ward 


TypeError: ('Bad input argument to theano function with name "<ipython-input-16-f8a13d93291b>:9"  at index 0(0-based)', "int() argument must be a string, a bytes-like object or a number, not 'map'")

In [40]:
seed = "thousands of years afterwards"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
 wards.
"Ankteroyease tick."
Thegruveny
 chote.
"Yom
i. lewwabrim nigher as the hlfaftHerePringist the will.
"Aforeh, as and bar can not u?"
Kyle
ngat ic the dide shat't mac, bung the Prince sunty more
mond
f"
ipKngsehfiate. said, aseming and you. AGaJd," said.
"Anle
rean nes. Arrleard
ilively to more, sh 
----


In [41]:
seed = "which bravely admits this to itself"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


----
 tself ad him. "? ullackinkt for allifaJking.
"
Thetures.
"Prince ta lang a sallike hav more?"
"I'me fime wgravilleIt herse. I'mqle. The
punult, mish be there guatteeh and's hou. "ken Arebed to be and more liugherd
to bicked bodet the t hou n'le clope clong.
"At saddy, sacked didinp bide
fack,
sstdruviall 
----
